In [63]:
# Import Libraries
import requests
import pandas as pd
from census import Census
from pathlib import Path
import matplotlib.pyplot as plt

In [64]:
# Use Path to open csvs.
laws_path = Path('Resources/CDC_Nutrition__Physical_Activity__and_Obesity_-_Legislation.csv')
deaths_path = Path('Resources/NCHS_-_Leading_Causes_of_Death__United_States.csv')
pop_path = Path('Resources/sc-est2017-alldata5.csv')

In [65]:
# Use Pandas to the read Legislature data.
laws_df = pd.read_csv(laws_path)

# Cleaning Data Set
laws_df = laws_df.drop(columns = ['Quarter', 'DataSource', 'Comments', 'EnactedDate', 'EffectiveDate', 
                                  'GeoLocation', 'PolicyTypeID', 'HealthTopicID', 'PolicyTopicID', 'Title',
                                 'SettingID', 'ProvisionID', 'Citation','StatusAltValue','DataType','DisplayOrder'])
laws_df = laws_df.rename(columns={'LocationAbbr': 'StateAbbr',
                                 'LocationDesc': 'State'})

laws_df.head()


,Year,StateAbbr,State,HealthTopic,PolicyTopic,Setting,Status
0,2009,CO,Colorado,Nutrition,Appropriations,School/After School,Enacted
1,2010,KY,Kentucky,Obesity,Food Restrictions,Community,Enacted
2,2010,KY,Kentucky,Physical Activity,Bicycling,Community,Enacted
3,2010,TX,Texas,Obesity,Sugar Sweetened Beverages,Early Care and Education,Enacted
4,2009,CO,Colorado,Nutrition,Agriculture and Farming,Community,Enacted


In [66]:
# Use Pandas to the read Legislature data
deaths_df = pd.read_csv(deaths_path)
deaths_df = deaths_df.rename(columns={'113 Cause Name': 'Full Cause Name',
                                     'State':'NAME'})
deaths_df['Year'] = deaths_df['Year'].astype('str')
deaths_df.dtypes

Year                        object
Full Cause Name             object
Cause Name                  object
NAME                        object
Deaths                       int64
Age-adjusted Death Rate    float64
dtype: object

In [67]:
# Grouping the population data frame to get sums for each year 
pop_df = pd.read_csv(pop_path)
pop_df = pop_df.drop(columns=['SUMLEV','REGION','DIVISION','STATE','SEX','ORIGIN','RACE','AGE','CENSUS2010POP','ESTIMATESBASE2010'])
grouped_pop = pop_df.groupby(['NAME'])

# Cleaning the grouped data frame
grouped_pop = grouped_pop[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014',
                          'POPESTIMATE2015','POPESTIMATE2016','POPESTIMATE2017']].sum()
grouped_pop = grouped_pop.rename(columns={'POPESTIMATE2010':'2010','POPESTIMATE2011':'2011','POPESTIMATE2012':'2012',
                                          'POPESTIMATE2013':'2013','POPESTIMATE2014':'2014','POPESTIMATE2015':'2015',
                                          'POPESTIMATE2016':'2016','POPESTIMATE2017':'2017'})

grouped_pop = grouped_pop.reset_index()


# Setting up a new data frame to be able to merge the population with the deaths
merge_pop = pd.melt(grouped_pop, id_vars=["NAME"], var_name='Year', value_name='Population')

merge_pop.dtypes
# merge_pop.head()

NAME          object
Year          object
Population     int64
dtype: object

In [72]:
merged_df = pd.merge(merge_pop,deaths_df, on=(['NAME','Year']),how='left')
merged_df['Death Percentage'] = (merged_df['Deaths'] / merged_df['Population']) * 100
merged_df


,NAME,Year,Population,Full Cause Name,Cause Name,Deaths,Age-adjusted Death Rate,Death Percentage
0,Alabama,2010,19418292,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,2394,49.6,0.012329
1,Alabama,2010,19418292,All Causes,All causes,48038,939.7,0.247385
2,Alabama,2010,19418292,Alzheimer's disease (G30),Alzheimer's disease,1523,31.2,0.007843
3,Alabama,2010,19418292,Cerebrovascular diseases (I60-I69),Stroke,2619,51.6,0.013487
4,Alabama,2010,19418292,Chronic lower respiratory diseases (J40-J47),CLRD,2866,55.4,0.014759
...,...,...,...,...,...,...,...,...
4483,Wyoming,2017,2369120,"Diseases of heart (I00-I09,I11,I13,I20-I51)",Heart disease,1001,148.9,0.042252
4484,Wyoming,2017,2369120,Influenza and pneumonia (J09-J18),Influenza and pneumonia,115,17.5,0.004854
4485,Wyoming,2017,2369120,"Intentional self-harm (suicide) (*U03,X60-X84,...",Suicide,157,26.9,0.006627
4486,Wyoming,2017,2369120,Malignant neoplasms (C00-C97),Cancer,948,136.1,0.040015


In [69]:
# Grouping the laws data frame to get counts
grouped_laws = laws_df.sort_values(by = ['State','HealthTopic'])
grouped_laws = grouped_laws.groupby(['State','Year','HealthTopic'])

laws_per_year = grouped_laws.count()
laws_per_year = laws_per_year.drop(columns=['StateAbbr', 'PolicyTopic','Setting'])
laws_per_year = laws_per_year.rename(columns={'Status':'Count'})
laws_per_year


Count
State   Year HealthTopic             
Alabama 2001 Nutrition              8
        2002 Nutrition              8
        2003 Nutrition              1
             Physical Activity      2
        2004 Nutrition              3
...                               ...
Wyoming 2013 Nutrition              5
             Obesity                1
             Physical Activity      1
        2016 Physical Activity      1
        2017 Nutrition              2

[1372 rows x 1 columns]

In [70]:
# Creating New Data Frame with state and Cause Name
